In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:02<00:00, 12723759.10it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 203502.90it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3757545.70it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 17963624.79it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:              #X is input data and y is corresponding labels
    print(f"Shape of X [N, C, H, W]: {X.shape}")  #N is batch size,C is channel 3 for rgb 1 for grayscale, H height, W weight
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
# Get cpu, gpu or mps device for training.
#automatically selects best available device for better performance
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()            # Calling parent class constructor
        self.flatten = nn.Flatten()   #input layer of nn. flattens the 28*28 pixel image into 1d 784 array
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 800),    # linear part of first hidden layer
            nn.ReLU(),                # non linearity added to the first hidden layer
            nn.Linear(800, 512),      #512 input neurons 512 output neurons
            nn.ReLU(),
            nn.Linear(512, 10)        #output layer with 10 classes of fashionmnist dataset
        )

    def forward(self, x):             #how data is passed
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=800, bias=True)
    (1): ReLU()
    (2): Linear(in_features=800, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):      #batch is batch index
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()  #computes the gradients
        optimizer.step()  #updates the model parameters
        optimizer.zero_grad() #clears the gradients of all optimized parameters to prepare for the next iteration.

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)  #gives the total number of samples processed up to and including the current batch.
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301562  [   64/60000]
loss: 0.525709  [ 6464/60000]
loss: 0.387602  [12864/60000]
loss: 0.472494  [19264/60000]
loss: 0.462032  [25664/60000]
loss: 0.425779  [32064/60000]
loss: 0.363856  [38464/60000]
loss: 0.533242  [44864/60000]
loss: 0.473249  [51264/60000]
loss: 0.475728  [57664/60000]
Test Error: 
 Accuracy: 84.0%, Avg loss: 0.426240 

Epoch 2
-------------------------------
loss: 0.286592  [   64/60000]
loss: 0.356743  [ 6464/60000]
loss: 0.257698  [12864/60000]
loss: 0.361727  [19264/60000]
loss: 0.461772  [25664/60000]
loss: 0.401063  [32064/60000]
loss: 0.309983  [38464/60000]
loss: 0.481428  [44864/60000]
loss: 0.392749  [51264/60000]
loss: 0.424855  [57664/60000]
Test Error: 
 Accuracy: 85.4%, Avg loss: 0.391965 

Epoch 3
-------------------------------
loss: 0.232208  [   64/60000]
loss: 0.337427  [ 6464/60000]
loss: 0.217497  [12864/60000]
loss: 0.300717  [19264/60000]
loss: 0.364806  [25664/60000]
loss: 0.347181  [32064/600

accuracy with 5 epochs: 64% [original code]<br>
with 10 epochs: 70.9% <br>
by adding one extra hidden layer: 65%<br>
changing optimizer to Adam : 87%


In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


<br>
<br>
<br>
<br>
<br>